In [1]:
import pandas as pd
import scipy.sparse as sparse
import numpy as np
from scipy.sparse.linalg import spsolve

In [2]:
from lightfm.evaluation import auc_score

/Users/Anna.Rudenko/opt/anaconda3/envs/myenv/lib/python3.8/site-packages/lightfm/_lightfm_fast.py:9: UserWarning: LightFM was compiled without OpenMP support. Only a single thread will be used.
  warnings.warn('LightFM was compiled without OpenMP support. '


In [3]:
from lightfm import LightFM
from lightfm.data import Dataset

In [4]:
retail_data = pd.read_csv('res_wo_ind.csv')
retail_data.head(5)

,CardHolder,ChequeId,CALDAY,Discount,SalesCount,SalesSumma,Material,Subcategory,net_weight,rpa_wgh1,rpa_wgh2,rpa_wgh3,rpa_wgh4,unit_of_wt,vendor,engname_rpa3,engname_rpa2,engname_rpa4,Class
0,10008254,1013172924,20200130,397.28,1.258,927.00,299994,130102,0.0,FOOD,13.0,1301,130102.0,G,NaN,Fresh salmon,Fresh and processed fish products,"Cut salmon, service",FOOD
1,13185016,493914559,20181116,174.99,0.982,755.35,299994,130102,0.0,FOOD,13.0,1301,130102.0,G,NaN,Fresh salmon,Fresh and processed fish products,"Cut salmon, service",FOOD
2,14790718,246904176,20180404,92.96,0.650,522.84,299994,130102,0.0,FOOD,13.0,1301,130102.0,G,NaN,Fresh salmon,Fresh and processed fish products,"Cut salmon, service",FOOD
3,14790718,246904334,20180404,81.33,0.570,458.68,299994,130102,0.0,FOOD,13.0,1301,130102.0,G,NaN,Fresh salmon,Fresh and processed fish products,"Cut salmon, service",FOOD
4,15088785,178450469,20180128,49.90,1.044,939.18,299994,130102,0.0,FOOD,13.0,1301,130102.0,G,NaN,Fresh salmon,Fresh and processed fish products,"Cut salmon, service",FOOD


In [23]:
np.random.seed(0)
pct_test = 0.2
num_samples = int(np.ceil(pct_test*len(nonzero_pairs)))

In [28]:
from scipy.stats import bernoulli
# import seaborn as sb

data_bern = bernoulli.rvs(size=20,p=0.8)

In [31]:
retail_data.shape

(995136, 19)

In [33]:
from scipy.stats import bernoulli
def train_test_samples(df, size=995136, p =0.2):
    data_bern = bernoulli.rvs(size=size,p=p)
    array_to_table = []
    for i in data_bern:
        if i == 1:
            array_to_table.append('test')
        else:
            array_to_table.append('train')
    df['eval_sample'] = array_to_table
            
            

In [34]:
retail_data_2 = retail_data
train_test_samples(retail_data_2)
retail_data_2.head()

,CardHolder,ChequeId,CALDAY,Discount,SalesCount,SalesSumma,Material,Subcategory,net_weight,rpa_wgh1,rpa_wgh2,rpa_wgh3,rpa_wgh4,unit_of_wt,vendor,engname_rpa3,engname_rpa2,engname_rpa4,Class,eval_sample
0,10008254,1013172924,20200130,397.28,1.258,927.00,299994,130102,0.0,FOOD,13.0,1301,130102.0,G,NaN,Fresh salmon,Fresh and processed fish products,"Cut salmon, service",FOOD,train
1,13185016,493914559,20181116,174.99,0.982,755.35,299994,130102,0.0,FOOD,13.0,1301,130102.0,G,NaN,Fresh salmon,Fresh and processed fish products,"Cut salmon, service",FOOD,train
2,14790718,246904176,20180404,92.96,0.650,522.84,299994,130102,0.0,FOOD,13.0,1301,130102.0,G,NaN,Fresh salmon,Fresh and processed fish products,"Cut salmon, service",FOOD,train
3,14790718,246904334,20180404,81.33,0.570,458.68,299994,130102,0.0,FOOD,13.0,1301,130102.0,G,NaN,Fresh salmon,Fresh and processed fish products,"Cut salmon, service",FOOD,train
4,15088785,178450469,20180128,49.90,1.044,939.18,299994,130102,0.0,FOOD,13.0,1301,130102.0,G,NaN,Fresh salmon,Fresh and processed fish products,"Cut salmon, service",FOOD,train


In [73]:
from scipy.sparse import coo_matrix

In [6]:

def get_user_list(df, user_column):    
    return np.sort(df[user_column].unique())

In [7]:
def get_item_list(df, item_name_column):
    
    
    item_list = df[item_name_column].unique()
    
    
    return item_list

In [21]:
def get_feature_list(df, listOfNames):
    result = pd.Series([])
    for i in listOfNames:
        result = pd.concat([result, df[i]], ignore_index = True)
    
    return result.unique()

In [14]:
s0 = pd.Series([])
s1 = pd.Series(['a', 'b'])
s2 = pd.Series(['c', 'd'])
s3 = pd.concat([s1, s2])

<ipython-input-14-8356a52b9f46>:1: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  s0 = pd.Series([])


In [22]:
def id_mappings(user_list, item_list, feature_list):

    user_to_index_mapping = {}
    index_to_user_mapping = {}
    for user_index, user_id in enumerate(user_list):
        user_to_index_mapping[user_id] = user_index
        index_to_user_mapping[user_index] = user_id
        
    item_to_index_mapping = {}
    index_to_item_mapping = {}
    for item_index, item_id in enumerate(item_list):
        item_to_index_mapping[item_id] = item_index
        index_to_item_mapping[item_index] = item_id
        
    feature_to_index_mapping = {}
    index_to_feature_mapping = {}
    for feature_index, feature_id in enumerate(feature_list):
        feature_to_index_mapping[feature_id] = feature_index
        index_to_feature_mapping[feature_index] = feature_id
        
        
    return user_to_index_mapping, index_to_user_mapping, \
           item_to_index_mapping, index_to_item_mapping, \
           feature_to_index_mapping, index_to_feature_mapping

In [60]:
def get_user_product_interaction(main_df):
    
    # creating a dataframe consists of TWO columns user_id, and product_name (product bought by the user) for the train data
    user_to_product_train_df = main_df[main_df["eval_sample"] == "train"][["CardHolder", "Material"]]
#     .\
#     merge(order_products_train_df[["order_id", "product_id"]]).merge(products_df[["product_id", "product_name"]])\
#     [["user_id", "product_name"]].copy()
    
    # giving rating as the number of product purchase count
    user_to_product_train_df["product_count"] = 1
    user_to_product_rating_train = user_to_product_train_df.groupby(["CardHolder", "Material"], as_index = False)["product_count"].sum()
    
    # creating a dataframe consists of TWO columns user_id, and product_name (product bought by the user) for the test data
    user_to_product_test_df = main_df[main_df["eval_sample"] == "test"][["CardHolder", "Material"]]
#     .\
#     merge(order_products_test_df[["order_id", "product_id"]]).merge(products_df[["product_id", "product_name"]])\
#     [["user_id", "product_name"]].copy()
    
    # giving rating as the number of product purchase count (including the previous purchase in the training data)
    user_to_product_test_df["product_count"] = 1
    user_to_product_rating_test = user_to_product_test_df.groupby(["CardHolder", "Material"], as_index = False)["product_count"].sum()
    
    # merging with the previous training user_to_product_rating_training
    
    user_to_product_rating_test = user_to_product_rating_test.\
    merge(user_to_product_rating_train.rename(columns = {"product_count" : "previous_product_count"}), how = "left").fillna(0)
    user_to_product_rating_test["product_count"] = user_to_product_rating_test.apply(lambda x: x["previous_product_count"] + \
                                                                                    x["product_count"], axis = 1)
    user_to_product_rating_test.drop(columns = ["previous_product_count"], inplace = True)
    
    return user_to_product_rating_train, user_to_product_rating_test

In [74]:
def get_interaction_matrix(df, df_column_as_row, df_column_as_col, df_column_as_value, row_indexing_map, 
                          col_indexing_map):
    
    row = df[df_column_as_row].apply(lambda x: row_indexing_map[x]).values
    col = df[df_column_as_col].apply(lambda x: col_indexing_map[x]).values
    value = df[df_column_as_value].values
    
    return coo_matrix((value, (row, col)), shape = (len(row_indexing_map), len(col_indexing_map)))

In [67]:
def get_product_feature_interaction(main_df, listOfNames, listOfWeights):
#     product_df, aisle_df, department_df, aisle_weight = 1, department_weight = 1):
#     item_feature_df = product_df.merge(aisle_df).merge(department_df)[["product_name", "aisle", "department"]]
    
    list_prolonged = ["Material"]
    list_prolonged.extend(listOfNames)
    item_feature_df = main_df[list_prolonged]
    # start indexing
#     item_feature_df["product_name"] = item_feature_df["product_name"]
#     item_feature_df["aisle"] = item_feature_df["aisle"]
#     item_feature_df["department"] = item_feature_df["department"]
    
    # allocate aisle and department into one column as "feature"
    feature_df = pd.Series([])
    for nameWeight in zip(listOfNames, listOfWeights):
        name = nameWeight[0]
        weight = nameWeight[1]
        product_feature_df = item_feature_df[["Material", name]].rename(columns = {name : "feature"})
        product_feature_df["feature_count"] = weight       
        feature_df = pd.concat([feature_df, product_feature_df], ignore_index=True)
#     product_aisle_df = item_feature_df[["product_name", "aisle"]].rename(columns = {"aisle" : "feature"})
#     product_aisle_df["feature_count"] = aisle_weight # adding weight to aisle feature
#     product_department_df = item_feature_df[["product_name", "department"]].rename(columns = {"department" : "feature"})
#     product_department_df["feature_count"] = department_weight # adding weight to department feature
    
#     # combining aisle and department into one
#     product_feature_df = pd.concat([product_aisle_df, product_department_df], ignore_index=True)
    
#     # saving some memory
#     del item_feature_df
#     del product_aisle_df
#     del product_department_df
    
    
    # grouping for summing over feature_count
    feature_df = feature_df.groupby(["Material", "feature"], as_index = False)["feature_count"].sum()
    
    
    return feature_df

In [57]:
users = get_user_list(retail_data_2, "CardHolder")
items = get_item_list(retail_data_2, "Material")
features = get_feature_list(retail_data_2, ["rpa_wgh1",  "engname_rpa2", "engname_rpa3", "engname_rpa4"])

<ipython-input-21-8e996483c728>:2: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  result = pd.Series([])


In [58]:
features

array(['FOOD', 'NONFOOD', 'Fresh and processed fish products', ...,
       '"Frozen meat by-products in branded package "', '"Rucksacks "',
       'Plastic suitcases'], dtype=object)

In [59]:
user_to_index_mapping, index_to_user_mapping, \
           item_to_index_mapping, index_to_item_mapping, \
           feature_to_index_mapping, index_to_feature_mapping = id_mappings(users, items, features)

In [62]:
listOfNames = ["rpa_wgh1",  "engname_rpa2", "engname_rpa3", "engname_rpa4"]
listOfWeights =  [1,4,2,3]

In [63]:
user_to_product_rating_train, user_to_product_rating_test = get_user_product_interaction(retail_data_2)

In [68]:
product_to_feature = get_product_feature_interaction(retail_data_2, listOfNames, listOfWeights)

<ipython-input-67-e1f70be8e557>:14: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  feature_df = pd.Series([])


In [69]:
user_to_product_rating_train.head()

,CardHolder,Material,product_count
0,10008254,299994,1
1,10011935,14228,1
2,10011935,18234,1
3,10011935,51291,1
4,10011935,51593,1


In [70]:
user_to_product_rating_test.head()

,CardHolder,Material,product_count
0,10011935,12276,1.0
1,10011935,12825,1.0
2,10011935,290306,1.0
3,10011935,299240,1.0
4,10011935,299997,1.0


In [71]:

product_to_feature.head()

,Material,feature,feature_count
0,11774.0,"""Hot beef meals, own production """,57.0
1,11774.0,FOOD,19.0
2,11774.0,"Hot dishes, own production",38.0
3,11774.0,Own production culinary products,76.0
4,11800.0,FOOD,107.0


In [75]:

# generate user_item_interaction_matrix for train data
user_to_product_interaction_train = get_interaction_matrix(user_to_product_rating_train, "CardHolder", 
                                                    "Material", "product_count", user_to_index_mapping, item_to_index_mapping)

# generate user_item_interaction_matrix for test data
user_to_product_interaction_test = get_interaction_matrix(user_to_product_rating_test, "CardHolder", 
                                                    "Material", "product_count", user_to_index_mapping, item_to_index_mapping)

# generate item_to_feature interaction
product_to_feature_interaction = get_interaction_matrix(product_to_feature, "Material", "feature",  "feature_count", 
                                                        item_to_index_mapping, feature_to_index_mapping)

In [76]:
user_to_product_interaction_train

<18977x7251 sparse matrix of type '<class 'numpy.int64'>'
	with 460231 stored elements in COOrdinate format>

In [77]:
user_to_product_interaction_test

<18977x7251 sparse matrix of type '<class 'numpy.float64'>'
	with 148208 stored elements in COOrdinate format>

In [78]:
product_to_feature_interaction 

<7251x1756 sparse matrix of type '<class 'numpy.float64'>'
	with 28954 stored elements in COOrdinate format>

In [221]:
model_without_features = LightFM(loss = "warp")

In [222]:
model_without_features.fit(user_to_product_interaction_train,
          user_features=None, 
          item_features=None, 
          sample_weight=None, 
          epochs=16, 
          num_threads=4,
          verbose=True)

Epoch 0
Epoch 1
Epoch 2
Epoch 3
Epoch 4
Epoch 5
Epoch 6
Epoch 7
Epoch 8
Epoch 9
Epoch 10
Epoch 11
Epoch 12
Epoch 13
Epoch 14
Epoch 15


In [223]:
auc_without_features = auc_score(model = model_without_features, 
                        test_interactions = user_to_product_interaction_test,
                        num_threads = 4, check_intersections = False)

In [224]:
print("average AUC without adding item-feature interaction = {0:.{1}f}".format(auc_without_features.mean(), 2))

average AUC without adding item-feature interaction = 0.90


In [84]:
model_with_features = LightFM(loss = "warp")
model_with_features.fit(user_to_product_interaction_train,
          user_features=None, 
          item_features=product_to_feature_interaction, 
          sample_weight=None, 
          epochs=1, 
          num_threads=4,
          verbose=False)

In [85]:
auc_with_features = auc_score(model = model_with_features, 
                        test_interactions = user_to_product_interaction_test,
                        train_interactions = user_to_product_interaction_train, 
                        item_features = product_to_feature_interaction,
                        num_threads = 4, check_intersections=False)

In [86]:
print("average AUC without adding item-feature interaction = {0:.{1}f}".format(auc_with_features.mean(), 2))

average AUC without adding item-feature interaction = 0.50


In [87]:
def combined_train_test(train, test):

    # initialising train dict
    train_dict = {}
    for train_row, train_col, train_data in zip(train.row, train.col, train.data):
        train_dict[(train_row, train_col)] = train_data
        
    # replacing with the test set
    
    for test_row, test_col, test_data in zip(test.row, test.col, test.data):
        train_dict[(test_row, test_col)] = max(test_data, train_dict.get((test_row, test_col), 0))
        
    
    # converting to the row
    row_element = []
    col_element = []
    data_element = []
    for row, col in train_dict:
        row_element.append(row)
        col_element.append(col)
        data_element.append(train_dict[(row, col)])
        
    # converting to np array
    
    row_element = np.array(row_element)
    col_element = np.array(col_element)
    data_element = np.array(data_element)
    
    return coo_matrix((data_element, (row_element, col_element)), shape = (train.shape[0], train.shape[1]))

In [89]:
user_to_product_interaction = combined_train_test(user_to_product_interaction_train, 
                                                 user_to_product_interaction_test)

In [229]:
class recommendation_sampling:
 
    def __init__(self, model, items = items, user_to_product_interaction_matrix = user_to_product_interaction, 
                user2index_map = user_to_index_mapping,  product_to_feature_3 =product_to_feature_3):
        
        self.user_to_product_interaction_matrix = user_to_product_interaction_matrix
        self.model = model
        self.items = items
        self.user2index_map = user2index_map
        self.product_to_feature =product_to_feature_3
    
    def recommendation_for_user(self, user):
        
        # getting the userindex
        
        userindex = self.user2index_map.get(user, None)
        
        if userindex == None:
            return None
        
        users = [userindex]
        
        # products already bought
        
        known_positives = self.items[self.user_to_product_interaction_matrix.tocsr()[userindex].indices]
#         print(known_positives.shape)
        known_positives_names = [product_to_feature_3.feature[product_to_feature_3['Material'] == i].tolist()[0] for i in known_positives]
    #[retail_data[retail_data.Material == i]['engname_rpa3'] for i in known_positives]
        # scores from model prediction
        scores = self.model.predict(user_ids = users, item_ids = np.arange(self.user_to_product_interaction_matrix.shape[1]))
        
        # top items
        
        top_items = self.items[np.argsort(-scores)]
#         print(top_items.shape)
        top_items_names = [product_to_feature_3.feature[product_to_feature_3['Material'] == i].tolist()[0] for i in top_items]
#         print(len(top_items_names[0]),len(top_items_names[1]))
        # printing out the result
        print("User %s" % user)
        print("     Known positives:")
        
        for x in known_positives_names[:7]:
            print("                  %s" % x)
            
            
        print("     Recommended:")
        
        for x in top_items_names[:7]:
            print("                  %s" % x)

In [230]:
recom = recommendation_sampling(model = model_without_features)

In [240]:
retail_data['CardHolder'][117]

11854166

In [241]:
recom.recommendation_for_user(11854166)

User 11854166
     Known positives:
                  Fresh salmon
                  Fresh salmon
                  Factory-made rye, rye and wheat bread
                  Own production bread
                  Baking own production
                  Gingerbread, round cracknels
                  Ice cream
     Recommended:
                  Fresh chicken
                  Berries
                  Exotic fruit
                  Greens
                  Fresh chicken
                  Chicken eggs
                  Fresh salmon


In [120]:
product_to_feature_3 = get_product_feature_interaction(retail_data, ["engname_rpa3"], [3])

<ipython-input-67-e1f70be8e557>:14: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  feature_df = pd.Series([])


In [142]:
product_to_feature_3.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 7267 entries, 0 to 7266
Data columns (total 3 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Material       7267 non-null   float64
 1   feature        7267 non-null   object 
 2   feature_count  7267 non-null   float64
dtypes: float64(2), object(1)
memory usage: 547.1+ KB


In [184]:
product_to_feature_3.feature[product_to_feature_3['Material'] == 299994].tolist()[0]

'Fresh salmon'

In [ ]:
retail_data_2[["Material"]]

In [52]:
retail_data_2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 995136 entries, 0 to 995135
Data columns (total 20 columns):
 #   Column        Non-Null Count   Dtype  
---  ------        --------------   -----  
 0   CardHolder    995136 non-null  int64  
 1   ChequeId      995136 non-null  int64  
 2   CALDAY        995136 non-null  int64  
 3   Discount      995136 non-null  float64
 4   SalesCount    995136 non-null  float64
 5   SalesSumma    995136 non-null  float64
 6   Material      995136 non-null  int64  
 7   Subcategory   995136 non-null  int64  
 8   net_weight    995136 non-null  float64
 9   rpa_wgh1      995136 non-null  object 
 10  rpa_wgh2      995136 non-null  float64
 11  rpa_wgh3      995136 non-null  int64  
 12  rpa_wgh4      995136 non-null  float64
 13  unit_of_wt    995136 non-null  object 
 14  vendor        910395 non-null  float64
 15  engname_rpa3  995136 non-null  object 
 16  engname_rpa2  995136 non-null  object 
 17  engname_rpa4  995136 non-null  object 
 18  Clas

In [48]:
item_lookup1 = retail_data_2[['Material', 'engname_rpa3']].rename(columns = {"engname_rpa3" : "feature"})
item_lookup2 = retail_data_2[['Material', 'engname_rpa4']].rename(columns = {"engname_rpa4" : "feature"})
product_feature_df = pd.concat([item_lookup1, item_lookup2], ignore_index=True)

In [49]:
product_feature_df 

,Material,feature
0,299994,Fresh salmon
1,299994,Fresh salmon
2,299994,Fresh salmon
3,299994,Fresh salmon
4,299994,Fresh salmon
...,...,...
1990267,463390,"""Rucksacks """
1990268,465374,Plastic suitcases
1990269,465390,Plastic suitcases
1990270,465388,Plastic suitcases


In [16]:
pd.concat([s1, s0],  ignore_index=True).unique()

array(['a', 'b'], dtype=object)

In [11]:
s3

0    a
1    b
0    c
1    d
dtype: object

In [20]:
def func(array):
    for i in array:
        print(i)
array = ['1', '2', '3', '4']
func(array)

1
2
3
4


In [ ]:
cleaned_retail = retail_data.loc[pd.isnull(retail_data.CardHolder) == False]
# cleaned_retail['SalesCount'] = cleaned_retail.SalesCount.astype(int) # Convert to int for customer ID
cleaned_retail_2 = cleaned_retail[['Material', 'SalesCount', 'CardHolder']] # Get rid of unnecessary info
grouped_cleaned = cleaned_retail_2.groupby(['CardHolder', 'Material']).sum().reset_index() # Group together
grouped_cleaned.SalesCount.loc[grouped_cleaned.SalesCount == 0] = 1 # Replace a sum of zero purchases with a one to
# indicate purchased
grouped_purchased = grouped_cleaned.query('SalesCount > 0')